In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('timeseries_data.csv')

In [3]:
# Filter the DataFrame to exclude rows with the year 2023
df_train = df[df['Season'] != '2023-08-01']

# Filter data for the 2023 season for testing
df_test = df[df['Season'] == '2023-08-01']

In [4]:
# Select features and target variable
features = ['GS', 'A', 'CS', 'BPS', 'MP', 'TSB(%)']
target = 'Pts'

# Split the data into features (X) and target variable (y)
X = df_train[features]
y = df_train[target]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)

# Use the scaler trained on the training set to transform the 2023 season data
X_test_scaled = scaler.transform(df_test[features])

# Create a Linear Regression model
model = LinearRegression()

# Train the model on historical data
model.fit(X_train_scaled, y)

# Make predictions on the 2023 season data
y_pred_2023 = model.predict(X_test_scaled)

# Actual target values for the 2023 season
y_actual_2023 = df_test[target]


In [5]:
mse = mean_squared_error(y_actual_2023, y_pred_2023)
r2 = r2_score(y_actual_2023, y_pred_2023)

# Display evaluation metrics
print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r2:.4f}")

Mean Squared Error: 18.3885
R-squared: 0.9726


In [6]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [13]:
# Create a DataFrame to display the results for the 2023 season
results_2023_df = pd.DataFrame({
    'Player Name': df_test['Player Name'],
    'Position': df_test['Position'],
    'Team': df_test['Team'],
    'Actual Pts': df_test['Pts'],
    'Predicted Pts': y_pred_2023.round().astype(int) # Use rounded and integer predicted points
})

# Reset the index of the DataFrame
results_2023_df = results_2023_df.reset_index(drop=True)

# Display the results for the 2023 season
results_2023_df

,Player Name,Position,Team,Actual Pts,Predicted Pts
0,Salah,MID,LIV,156,160
1,Son,MID,TOT,136,138
2,Watkins,FWD,AVL,120,141
3,Haaland,FWD,MCI,112,141
4,Bowen,MID,WHU,111,110
...,...,...,...,...,...
768,Nwaneri,MID,ARS,0,1
769,Orford,MID,WHU,0,1
770,Casey,DEF,WHU,0,1
771,Bettinelli,GKP,CHE,-1,1


In [8]:
# Reset the options to their default values if needed
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [14]:
# Find the index of the row with the maximum 'Actual Pts' for each team
max_predicted_pts_indices = results_2023_df.groupby('Team')['Predicted Pts'].idxmax()

# Retrieve the corresponding rows using the indices
max_predicted_pts_rows = results_2023_df.loc[max_predicted_pts_indices]

max_predicted_pts_rows = max_predicted_pts_rows.reset_index(drop=True)

max_predicted_pts_rows

,Player Name,Position,Team,Actual Pts,Predicted Pts
0,Saka,MID,ARS,107,127
1,Watkins,FWD,AVL,120,141
2,Gross,MID,BHA,82,77
3,Solanke,FWD,BOU,98,110
4,Mbeumo,MID,BRE,87,78
5,Trafford,GKP,BUR,57,58
6,Palmer,MID,CHE,92,95
7,Andersen,DEF,CRY,69,76
8,A.Doucoure,MID,EVE,76,72
9,Leno,GKP,FUL,71,72
